In [1]:
#r "./../../../public/src/L4-application/BoSSSpad/bin/Release/net6.0/BoSSSpad.dll"
using System;
using System.Threading;
using System.Diagnostics;
using ilPSP;
using ilPSP.Utils;
using BoSSS.Platform;
using BoSSS.Foundation;
using BoSSS.Foundation.XDG;
using BoSSS.Foundation.Grid;
using BoSSS.Solution;
using BoSSS.Application.XNSE_Solver;
using BoSSS.Application.BoSSSpad;
using BoSSS.Foundation.Grid.Classic;
using BoSSS.Foundation.IO;
using BoSSS.Solution.AdvancedSolvers;
using BoSSS.Solution.Control;
using BoSSS.Solution.XNSECommon;
using BoSSS.Solution.NSECommon;
using BoSSS.Solution.LoadBalancing;
using BoSSS.Solution.LevelSetTools;
using BoSSS.Solution.XdgTimestepping;
using BoSSS.Solution.Utils;
using System.IO;
using BoSSS.Application.XNSE_Solver.PhysicalBasedTestcases;
using static BoSSS.Application.BoSSSpad.BoSSSshell;
using MomentFittingVariants = BoSSS.Foundation.XDG.XQuadFactoryHelper.MomentFittingVariants;
Init();

In [2]:
//defaults
string dbname = System.Environment.GetEnvironmentVariable("DATABASE_NAME");
string buildname = System.Environment.GetEnvironmentVariable("JOB_NAME");
buildname = String.IsNullOrEmpty(buildname)? "CollidingSpheres2D" : buildname; //Popcorn2D  VanishingSphere2D
dbname = String.IsNullOrEmpty(dbname)? "condStudyBDF2" : dbname; 
string table_name = String.Concat(buildname, "_", dbname);

In [3]:
using BoSSS.Solution.Gnuplot;


In [4]:
//ExecutionQueues
var myBatch = ExecutionQueues[4];
string WFlowName = table_name;
BoSSS.Application.BoSSSpad.BoSSSshell.WorkflowMgm.Init(WFlowName,myBatch);
BoSSS.Application.BoSSSpad.BoSSSshell.WorkflowMgm.SetNameBasedSessionJobControlCorrelation();
var myDB = BoSSS.Application.BoSSSpad.BoSSSshell.WorkflowMgm.DefaultDatabase; 
myDB

{ Session Count = 18; Grid Count = 2; Path = \\dc3\userspace\toprak\BoSSS-ConditionFinal\CollidingSpheres2D_condStudyBDF2 }

In [5]:
myDB.Sessions.Where(s => s.SuccessfulTermination == true)

#0: CollidingSpheres2D_condStudyBDF2	SolverFalse_J2048_k1_agg0_AMRFalse_LBFalse_vel8.33_r0.6_2D_NSE_rateOfRadius13.89_BDF2_ts80_condStudy_ct1	9/6/2024 5:16:46 PM	be212a66...
#1: CollidingSpheres2D_condStudyBDF2	SolverFalse_J2048_k3_agg0.3_AMRFalse_LBFalse_vel8.33_r0.6_2D_NSE_rateOfRadius13.89_BDF2_ts80_condStudy_ct1	9/6/2024 6:02:04 PM	3a1277db...
#2: CollidingSpheres2D_condStudyBDF2	SolverFalse_J2048_k3_agg0.2_AMRFalse_LBFalse_vel8.33_r0.6_2D_NSE_rateOfRadius13.89_BDF2_ts80_condStudy_ct1	9/6/2024 6:01:30 PM	4fa9b7ca...
#3: CollidingSpheres2D_condStudyBDF2	SolverFalse_J2048_k3_agg0.5_AMRFalse_LBFalse_vel8.33_r0.6_2D_NSE_rateOfRadius13.89_BDF2_ts80_condStudy_ct1	9/6/2024 6:17:26 PM	31298b47...
#4: CollidingSpheres2D_condStudyBDF2	SolverFalse_J2048_k3_agg0.1_AMRFalse_LBFalse_vel8.33_r0.6_2D_NSE_rateOfRadius13.89_BDF2_ts80_condStudy_ct1	9/6/2024 6:00:34 PM	587e1163...
#5: CollidingSpheres2D_condStudyBDF2	SolverFalse_J2048_k3_agg0.4_AMRFalse_LBFalse_vel8.33_r0.6_2D_NSE_rateOfRadius13.89_BD

In [6]:
BoSSSshell.WorkflowMgm.condReader.Update();

Session: be212a66-a95d-4bc0-ac59-c057342d4ff7
#timestep, physicalTime, TotCondNo-Vars0.1.2, MassMtxCondNo, 
Session: 3a1277db-c031-43c8-9f28-2664e201acf5
Session: 4fa9b7ca-a8ac-4371-95a2-7a3cadeed238
Session: 31298b47-06af-4233-bc8a-bb0c711d3581
Session: 587e1163-a13e-4e21-9f90-767bc2e5b815
Session: 24483630-4501-402c-b843-f4bf1a8fdace
Session: 84d1f11a-80f1-4e94-8db9-e629285d8c74
Session: 0c8e9995-17cf-4d84-b43d-90d5fd9f59c8
Session: 165ed84c-8de4-4712-bb0e-55e817a0becb
Session: 34587ff7-a7e6-42ba-b8cb-874ab68a80b8
Session: bc5dc74f-b3b1-447f-aefc-e1a4f27ac3a6
Session: a0abe1de-5001-4463-9158-dfef0f251ffc
Session: 4d86cba5-9634-429c-95fc-395dfda8c2ea
Session: 823916f3-5f9a-4e46-83dc-48476ea1a02a
Session: ba3aff2e-ddb8-4245-a22f-09238790deec
Session: f2383abe-fbf5-4300-a0a7-c6ff8e64a911
Session: 1a4b59e8-653f-4755-8cad-40d0b8f5b027
Session: d6eae438-3b92-4387-9ee0-c05cbafc2e0c
Session: be212a66-a95d-4bc0-ac59-c057342d4ff7
#timestep, physicalTime, TotCondNo-Vars0.1.2, MassMtxCondNo, 
Th

In [7]:
var Tab = BoSSSshell.WorkflowMgm.SessionTable;

In [8]:
var Tab2 = Tab.ExtractColumns( "AgglomerationThreshold",  "DGdegree:Velocity*","Grid:hMin","Grid:NoOfCells","#timestep", "MassMtxCondNo", "TotCondNo-Vars0.1.2");
Tab2.SaveToFile(table_name + $"_condSummary.json");
Tab2.Print()

    AgglomerationThreshold DGdegree:Velocity* Grid:hMin Grid:NoOfCells #timestep                                        MassMtxCondNo                                    TotCondNo-Vars0.1.2                              
0:  0                      1                  0.0625    2048           System.Collections.Generic.List`1[System.Double] System.Collections.Generic.List`1[System.Double] System.Collections.Generic.List`1[System.Double] 
1:  0.3                    3                  0.0625    2048           System.Collections.Generic.List`1[System.Double] System.Collections.Generic.List`1[System.Double] System.Collections.Generic.List`1[System.Double] 
2:  0.2                    3                  0.0625    2048           System.Collections.Generic.List`1[System.Double] System.Collections.Generic.List`1[System.Double] System.Collections.Generic.List`1[System.Double] 
3:  0.5                    3                  0.0625    2048           System.Collections.Generic.List`1[System.Double] Syst

In [9]:
using BoSSS.Solution.Gnuplot;
using System.Linq;

// arrange line color and types
var allColors = Enum.GetValues(typeof(LineColors)).Cast<LineColors>().ToArray();
var allPointTypes = Enum.GetValues(typeof(PointTypes)).Cast<PointTypes>().ToArray();
PointTypes[] myPointTypes = new PointTypes[]{ PointTypes.Diamond, PointTypes.Box,  PointTypes.LowerTriangle,PointTypes.OpenLowerTriangle };


Condition numbers for mass matrix

In [10]:
string xVarName = "#timestep";
string yVarName = "MassMtxCondNo";
int dgDegree = 1;

// Filter rows depending on selection
var Tab3 = Tab2.ExtractRows((iRow,RowEntries)=> 
                //Convert.ToDouble(RowEntries["AgglomerationThreshold"]) != 0 &&
                Convert.ToDouble(RowEntries["DGdegree:Velocity*"]) == dgDegree
            );

// Sort in ascending order of AgglomerationThreshold
System.Data.DataView view = Tab3.DefaultView;
view.Sort = "AgglomerationThreshold ASC"; 
Tab3 = view.ToTable();

Tab3.Print()


   AgglomerationThreshold DGdegree:Velocity* Grid:hMin Grid:NoOfCells #timestep                                        MassMtxCondNo                                    TotCondNo-Vars0.1.2                              
0: 0                      1                  0.0625    2048           System.Collections.Generic.List`1[System.Double] System.Collections.Generic.List`1[System.Double] System.Collections.Generic.List`1[System.Double] 
1: 0.1                    1                  0.0625    2048           System.Collections.Generic.List`1[System.Double] System.Collections.Generic.List`1[System.Double] System.Collections.Generic.List`1[System.Double] 
2: 0.2                    1                  0.0625    2048           System.Collections.Generic.List`1[System.Double] System.Collections.Generic.List`1[System.Double] System.Collections.Generic.List`1[System.Double] 
3: 0.3                    1                  0.0625    2048           System.Collections.Generic.List`1[System.Double] System.Co

In [11]:

Plot2Ddata plot = new Plot2Ddata();
    int i = 0;
plot.dataGroups = new Plot2Ddata.XYvalues[Tab3.Rows.Count];

foreach (System.Data.DataRow row in Tab3.Rows){
    //row.
    //*Console.WriteLine(row.Summary());
    double aggThre = (double)row.ItemArray[Tab2.Columns.IndexOf("AgglomerationThreshold")];
    var xValue = (List<double>)row.ItemArray[Tab2.Columns.IndexOf(xVarName)];
    var yValue = (List<double>)row.ItemArray[Tab2.Columns.IndexOf(yVarName)];
    Plot2Ddata.XYvalues xyGroup = new Plot2Ddata.XYvalues("AggThreshold="+aggThre, xValue, yValue);
    plot.dataGroups[i] = xyGroup;
    //ArrayTools.AddToArray(xyGroup, ref plot.dataGroups);
            var dataGroup = plot.dataGroups[i];
            dataGroup.Format.LineColor = allColors[i % allColors.Length];
            dataGroup.Format.PointType = myPointTypes[i % myPointTypes.Length];
            i++;
}

In [12]:
plot.PlotNow()

Using gnuplot: C:\Program Files (x86)\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
Note: In a Jupyter Worksheet, you must NOT have a trailing semicolon in order to see the plot on screen; otherwise, the output migth be surpressed.!


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 1x10 8 
 
 
 
 
 2x10 8 
 
 
 
 
 3x10 8 
 
 
 
 
 4x10 8 
 
 
 
 
 5x10 8 
 
 
 
 
 6x10 8 
 
 
 
 
 7x10 8 
 
 
 
 
 8x10 8 
 
 
 
 
 9x10 8 
 
 
 
 
 0 
 
 
 
 
 10 
 
 
 
 
 20 
 
 
 
 
 30 
 
 
 
 
 40 
 
 
 
 
 50 
 
 
 
 
 60 
 
 
 
 
 70 
 
 
 
 
 80 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 AggThreshold=0 
 
 
 AggThreshold=0 
 
 
 
	<path stroke='rgb(255, 0, 0)' d='M499.7,57.1 L553.1,57.1 M70.8,563.8 L79.4,393.8 L88.0,395.0 L96.6,563.8 L105.2,459.2 L113.8,554.5
 M131.0,541.9 L139.6,558.6 L148.2,65.5 L156.8,509.1 L165.4,563.7 L174.0,562.3 L182.6,365.6 L191.2,563.9
 L199.8,364.3 L208.4,562.3 L217.0,563.8 L225.6,557.4 L234.2,562.1 L242.8,558.6 L251.4,542.2 L260.0,563.6
 L268.6,558.5 L277.2,535.8 L285.8,563.8 L294.4,404.2 L303.0,562.4 L311.6,563.9 L320.2,492.9 L328.8,554.5
 M346.0,560.8 L354.6,563.2 L363.2,79.4 L371.8,509.1 L380.4,563.7 L389.0,562.5 L397.6,560.0 L406.1,563.9
 L414.7,560.0 L423.3,562.5 L431.9,563.7 L440.5,508.3 L449.1,69.9 L457.7,563.2 L466.3,560.7 M483.5,554.3
 L492.1,492.2 L500.7,563.9 L509.3,562.4 L517.9,402.8 L526.5,563.8 L535.1,535.7 L543.7,558.3 L552.3,563.6
 L560.9,541.9 L569.5,558.6 L578.1,562.0 L586.7,557.4 L595.3,563.8 L603.9,562.3 L612.5,365.6 L621.1,563.9
 L629.7,364.3 L638.3,562.3 L646.9,563.7 L655.5,508.3 L664.1,69.9 L672.7,558.6 L681.3,542.2 M698.5,554.3
 L707.1,458.8 L715.7,563.8 L724.3,393.8 L732.9,395.0 L741.5,563.8 L750.1,459.2 '/> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 AggThreshold=0.1 
 
 
 AggThreshold=0.1 
 
 
 
	<path stroke='rgb( 0, 255, 0)' d='M499.7,81.1 L553.1,81.1 M70.8,563.9 L79.4,563.9 L88.0,563.9 L96.6,563.9 L105.2,563.9 L113.8,563.9
 L122.4,563.9 L131.0,563.9 L139.6,563.9 L148.2,563.9 L156.8,563.9 L165.4,563.9 L174.0,563.9 L182.6,563.9
 L191.2,563.9 L199.8,563.9 L208.4,563.9 L217.0,563.9 L225.6,563.9 L234.2,563.9 L242.8,563.9 L251.4,564.0
 L260.0,563.9 L268.6,563.9 L277.2,563.9 L285.8,563.9 L294.4,563.9 L303.0,563.9 L311.6,563.9 L320.2,563.9
 L328.8,563.9 L337.4,563.9 L346.0,564.0 L354.6,564.0 L363.2,563.9 L371.8,563.9 L380.4,563.9 L389.0,563.9
 L397.6,563.9 L406.1,563.9 L414.7,563.9 L423.3,563.9 L431.9,563.9 L440.5,563.9 L449.1,563.9 L457.7,564.0
 L466.3,563.9 L474.9,563.9 L483.5,563.9 L492.1,563.9 L500.7,563.9 L509.3,563.9 L517.9,563.9 L526.5,563.9
 L535.1,563.9 L543.7,563.9 L552.3,563.9 L560.9,564.0 L569.5,563.9 L578.1,563.9 L586.7,563.9 L595.3,563.9
 L603.9,563.9 L612.5,563.9 L621.1,563.9 L629.7,563.9 L638.3,563.9 L646.9,563.9 L655.5,563.9 L664.1,563.9
 L672.7,563.9 L681.3,563.9 L689.9,563.9 L698.5,563.9 L707.1,563.9 L715.7,563.9 L724.3,563.9 L732.9,563.9
 L741.5,563.9 L750.1,563.9 '/> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 AggThreshold=0.2 
 
 
 AggThreshold=0.2 
 
 
 
	<path stroke='rgb( 0, 0, 255)' d='M499.7,105.1 L553.1,105.1 M70.8,564.0 L79.4,563.9 L88.0,563.9 L96.6,564.0 L105.2,564.0 L113.8,564.0
 L122.4,564.0 L131.0,563.9 L139.6,564.0 L148.2,563.9 L156.8,564.0 L165.4,564.0 L174.0,563.9 L182.6,564.0
 L191.2,564.0 L199.8,563.9 L208.4,564.0 L217.0,564.0 L225.6,564.0 L234.2,563.9 L242.8,564.0 L251.4,564.0
 L260.0,564.0 L268.6,564.0 L277.2,564.0 L285.8,564.0 L294.4,564.0 L303.0,563.9 L311.6,564.0 L320.2,564.0
 L328.8,564.0 L337.4,564.0 L346.0,564.0 L354.6,564.0 L363.2,564.0 L371.8,564.0 L380.4,564.0 L389.0,564.0
 L397.6,564.0 L406.1,564.0 L414.7,564.0 L423.3,564.0 L431.9,564.0 L440

In [13]:
plot.SaveToGIF(yVarName + "_k" + dgDegree + "_2D.png");
plot.SavePgfplotsFile(yVarName + "_k" + dgDegree + "_2D.tex");

Using gnuplot: C:\Program Files (x86)\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe


Condition numbers for operator matrix

In [14]:
string xVarName = "#timestep";
string yVarName = "TotCondNo-Vars0.1.2";

// Filter rows depending on selection
var Tab3 = Tab2.ExtractRows((iRow,RowEntries)=> 
                //Convert.ToDouble(RowEntries["AgglomerationThreshold"]) != 0 &&
                Convert.ToDouble(RowEntries["DGdegree:Velocity*"]) == dgDegree
            );

// Sort in ascending order of AgglomerationThreshold
System.Data.DataView view = Tab3.DefaultView;
view.Sort = "AgglomerationThreshold ASC"; 
Tab3 = view.ToTable();


In [15]:

Plot2Ddata plot = new Plot2Ddata();
    int i = 0;
plot.dataGroups = new Plot2Ddata.XYvalues[Tab3.Rows.Count];
foreach (System.Data.DataRow row in Tab3.Rows){
    //row.
    //*Console.WriteLine(row.Summary());
    double aggThre = (double)row.ItemArray[Tab2.Columns.IndexOf("AgglomerationThreshold")];
    var xValue = (List<double>)row.ItemArray[Tab2.Columns.IndexOf(xVarName)];
    var yValue = (List<double>)row.ItemArray[Tab2.Columns.IndexOf(yVarName)];
    Plot2Ddata.XYvalues xyGroup = new Plot2Ddata.XYvalues("AggThreshold="+aggThre, xValue, yValue);
    plot.dataGroups[i] = xyGroup;
    //ArrayTools.AddToArray(xyGroup, ref plot.dataGroups);
            var dataGroup = plot.dataGroups[i];
            dataGroup.Format.LineColor = allColors[i % allColors.Length];
            dataGroup.Format.PointType = myPointTypes[i % myPointTypes.Length];
            i++;
}

In [16]:
plot.PlotNow()

Using gnuplot: C:\Program Files (x86)\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
Note: In a Jupyter Worksheet, you must NOT have a trailing semicolon in order to see the plot on screen; otherwise, the output migth be surpressed.!


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 2x10 19 
 
 
 
 
 4x10 19 
 
 
 
 
 6x10 19 
 
 
 
 
 8x10 19 
 
 
 
 
 1x10 20 
 
 
 
 
 1.2x10 20 
 
 
 
 
 0 
 
 
 
 
 10 
 
 
 
 
 20 
 
 
 
 
 30 
 
 
 
 
 40 
 
 
 
 
 50 
 
 
 
 
 60 
 
 
 
 
 70 
 
 
 
 
 80 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 AggThreshold=0 
 
 
 AggThreshold=0 
 
 
 
	<path stroke='rgb(255, 0, 0)' d='M499.7,57.1 L553.1,57.1 M95.4,564.0 L103.7,564.0 L112.0,564.0 L120.3,564.0 L128.5,564.0 L136.8,564.0
 L145.1,530.9 L153.4,564.0 L161.7,564.0 L170.0,564.0 L178.3,564.0 L186.6,564.0 L194.8,564.0 L203.1,564.0
 L211.4,564.0 L219.7,564.0 L228.0,564.0 L236.3,564.0 L244.6,564.0 L252.9,564.0 L261.1,564.0 L269.4,564.0
 L277.7,564.0 L286.0,564.0 L294.3,564.0 L302.6,564.0 L310.9,564.0 L319.2,564.0 L327.4,564.0 L335.7,564.0
 L344.0,564.0 L352.3,100.8 L360.6,564.0 L368.9,564.0 L377.2,564.0 L385.5,564.0 L393.7,564.0 L402.0,564.0
 L410.3,564.0 L418.6,564.0 L426.9,564.0 L435.2,564.0 L443.5,564.0 L451.8,564.0 L460.0,564.0 L468.3,564.0
 L476.6,564.0 L484.9,450.9 L493.2,564.0 L501.5,564.0 L509.8,564.0 L518.1,564.0 L526.3,564.0 L534.6,564.0
 L542.9,564.0 L551.2,564.0 L559.5,564.0 L567.8,564.0 L576.1,564.0 L584.4,564.0 L592.6,564.0 L600.9,564.0
 L609.2,564.0 L617.5,564.0 L625.8,564.0 L634.1,564.0 L642.4,564.0 L650.7,564.0 L658.9,564.0 L667.2,564.0
 L675.5,564.0 L683.8,564.0 L692.1,528.3 L700.4,564.0 L708.7,564.0 L717.0,564.0 L725.2,564.0 L733.5,564.0
 L741.8,564.0 L750.1,564.0 '/> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 AggThreshold=0.1 
 
 
 AggThreshold=0.1 
 
 
 
	<path stroke='rgb( 0, 255, 0)' d='M499.7,81.1 L553.1,81.1 M95.4,564.0 L103.7,564.0 L112.0,564.0 L120.3,564.0 L128.5,564.0 L136.8,564.0
 L145.1,564.0 L153.4,564.0 L161.7,564.0 L170.0,564.0 L178.3,564.0 L186.6,564.0 L194.8,564.0 L203.1,564.0
 L211.4,564.0 L219.7,564.0 L228.0,564.0 L236.3,564.0 L244.6,564.0 L252.9,564.0 L261.1,564.0 L269.4,564.0
 L277.7,564.0 L286.0,564.0 L294.3,564.0 L302.6,564.0 L310.9,564.0 L319.2,564.0 L327.4,564.0 L335.7,564.0
 L344.0,564.0 L352.3,564.0 L360.6,564.0 L368.9,564.0 L377.2,564.0 L385.5,564.0 L393.7,564.0 L402.0,564.0
 L410.3,564.0 L418.6,564.0 L426.9,564.0 L435.2,564.0 L443.5,564.0 L451.8,564.0 L460.0,564.0 L468.3,564.0
 L476.6,564.0 L484.9,564.0 L493.2,564.0 L501.5,564.0 L509.8,564.0 L518.1,564.0 L526.3,564.0 L534.6,564.0
 L542.9,564.0 L551.2,564.0 L559.5,564.0 L567.8,564.0 L576.1,564.0 L584.4,564.0 L592.6,564.0 L600.9,564.0
 L609.2,564.0 L617.5,564.0 L625.8,564.0 L634.1,564.0 L642.4,564.0 L650.7,564.0 L658.9,564.0 L667.2,564.0
 L675.5,564.0 L683.8,564.0 L692.1,564.0 L700.4,564.0 L708.7,564.0 L717.0,564.0 L725.2,564.0 L733.5,564.0
 L741.8,564.0 L750.1,564.0 '/> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 AggThreshold=0.2 
 
 
 AggThreshold=0.2 
 
 
 
	<path stroke='rgb( 0, 0, 255)' d='M499.7,105.1 L553.1,105.1 M95.4,564.0 L103.7,564.0 L112.0,564.0 L120.3,564.0 L128.5,564.0 L136.8,564.0
 L145.1,564.0 L153.4,564.0 L161.7,564.0 L170.0,564.0 L178.3,564.0 L186.6,564.0 L194.8,564.0 L203.1,564.0
 L211.4,564.0 L219.7,564.0 L228.0,564.0 L236.3,564.0 L244.6,564.0 L252.9,564.0 L261.1,564.0 L269.4,564.0
 L277.7,564.0 L286.0,564.0 L294.3,564.0 L302.6,564.0 L310.9,564.0 L319.2,564.0 L327.4,564.0 L335.7,564.0
 L344.0,564.0 L352.3,564.0 L360.6,564.0 L368.9,564.0 L377.2,564.0 L385.5,564.0 L393.7,564.0 L402.0,564.0
 L410.3,564.0 L418.6,564.0 L426.9,564.0 L435.2,564.0 L443.5,564.0 L45

In [17]:
plot.SaveToGIF(yVarName + "_k" + dgDegree + "_2D.png");
plot.SavePgfplotsFile(yVarName + "_k" + dgDegree + "_2D.tex");

Using gnuplot: C:\Program Files (x86)\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
